In [2]:
from PIL import Image
from math import floor
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from pynq import allocate, Overlay
from random import seed
from random import randint
from random import getrandbits

In [3]:
def run_kernel():
    dma.sendchannel.transfer(in_buffer)
    dma.recvchannel.transfer(out_buffer)
    dma.sendchannel.wait()
    dma.recvchannel.wait()

In [4]:
filterBase = 24
dataBase = 60
def output_registers():
    print("Control Enable:         %d" % (controller.read(0)))
    print("Reset:                  %d" % (controller.read(4)))
    print("State Machine Register: %d" % (controller.read(8)))
    print("Last Calculated Value:  %d" % (controller.read(12)))
    print("Image Width:            %d" % (controller.read(16)))
    print("Image Height:           %d" % (controller.read(20)))
    print("\nFilter Set:")
    for i in range(9):
        print("filterSet[%d] = %d" % (i,controller.read((i*4)+filterBase)))
    print("\nData Set:")
    for i in range(9):
        print("dataSet[%d] = %d" % (i,controller.read((i*4)+dataBase)))
    print("\n")

In [10]:
Convolution_design = Overlay("Hardware-Files/PR_Convolution.bit",)
dfx = Convolution_design.dfx_controller_0
dma = Convolution_design.axi_dma_0
controller = Convolution_design.Convolution_Controll_0

In [11]:
width = 4
height = 4
kernel_size = 3

in_buffer = allocate(shape=(height, width, 1),dtype=np.uint32, cacheable=1)
out_buffer = allocate(shape=(height-2, width-2, 1),dtype=np.uint32, cacheable=1)

for i in range(width):
    for j in range(height):
        in_buffer[i][j] = i*width+j
#         in_buffer[j][i] = 10 if bool(getrandbits(1)) else 100

In [12]:
#Enable IP control register
controller.write(0,1)
kernelsize = 3
# High-pass kernel
kernel = [[  0,   0,   0 ],
          [  0,   1,   0 ],
          [  0,   0,   0 ]]

#Input Filter set
for x in range(kernelsize):
    for y in range(kernelsize):
        controller.write(filterBase+((x*kernelsize)+y)*4,kernel[y][x])

#Input width/height
controller.write(16,width)#Width
controller.write(20,height)#Height

In [13]:
output_registers()

Control Enable:         1
Reset:                  0
State Machine Register: 1
Last Calculated Value:  0
Image Width:            4
Image Height:           4

Filter Set:
filterSet[0] = 0
filterSet[1] = 0
filterSet[2] = 0
filterSet[3] = 0
filterSet[4] = 1
filterSet[5] = 0
filterSet[6] = 0
filterSet[7] = 0
filterSet[8] = 0

Data Set:
dataSet[0] = 0
dataSet[1] = 0
dataSet[2] = 0
dataSet[3] = 0
dataSet[4] = 0
dataSet[5] = 0
dataSet[6] = 0
dataSet[7] = 0
dataSet[8] = 0




In [14]:
run_kernel()

In [35]:
output_registers()

Control Enable:         1
Reset:                  0
State Machine Register: 1
Last Calculated Value:  9
Image Width:            8
Image Height:           8

Filter Set:
filterSet[0] = 0
filterSet[1] = 0
filterSet[2] = 0
filterSet[3] = 0
filterSet[4] = 1
filterSet[5] = 0
filterSet[6] = 0
filterSet[7] = 0
filterSet[8] = 0

Data Set:
dataSet[0] = 0
dataSet[1] = 0
dataSet[2] = 0
dataSet[3] = 0
dataSet[4] = 0
dataSet[5] = 0
dataSet[6] = 0
dataSet[7] = 0
dataSet[8] = 0




In [15]:
for i in range(height):
    for j in range(width):
        print(hex(in_buffer[i,j,0]),end=' ')
        if(j==width-1):
            print()

0x0 0x1 0x2 0x3 
0x4 0x5 0x6 0x7 
0x8 0x9 0xa 0xb 
0xc 0xd 0xe 0xf 


In [37]:
for i in range(height-2):
    for j in range(width-2):
        print(hex(out_buffer[i,j,0]),end=' ')
        if(j==width-3):
            print()

0x0 0x0 0x0 0x0 0x0 0x0 
0x0 0x0 0x0 0x0 0x0 0x0 
0x0 0x0 0x0 0x0 0x0 0x0 
0x0 0x0 0x0 0x0 0x0 0x0 
0x0 0x0 0x0 0x0 0x0 0x0 
0x0 0x0 0x0 0x0 0x0 0x0 


In [32]:
from pprint import pprint
pprint(Convolution_design.ip_dict)

{'Convolution_Controll_0': {'addr_range': 65536,
                            'device': <pynq.pl_server.device.XlnkDevice object at 0xaede96f0>,
                            'driver': <class 'pynq.overlay.DefaultIP'>,
                            'fullpath': 'Convolution_Controll_0',
                            'gpio': {},
                            'interrupts': {},
                            'mem_id': 's_axi_CTRL',
                            'parameters': {'AXI_ADDR_WIDTH': '10',
                                           'AXI_BUS_WIDTH': '32',
                                           'BRAM_WIDTH': '5',
                                           'CTRL_REG_ADDR_WIDTH': '7',
                                           'CTRL_REG_SIZE': '96',
                                           'C_BASEADDR': '0x43C10000',
                                           'C_HIGHADDR': '0x43C1FFFF',
                                           'Component_Name': 'Convolution_Controller_Convolution_Controll_

                                                              'description': 'MM2S '
                                                                             'DMA '
                                                                             'Control '
                                                                             'Register'},
                                                       'Reset': {'access': 'read-write',
                                                                 'bit_offset': 2,
                                                                 'bit_width': 1,
                                                                 'description': 'MM2S '
                                                                                'DMA '
                                                                                'Control '
                                                                                'Register'}},
                                         

                                                   'description': 'MM2S DMA '
                                                                  'Tail '
                                                                  'Descriptor '
                                                                  'Pointer '
                                                                  'Register',
                                                   'fields': {'Tail_Descriptor_Pointer': {'access': 'read-write',
                                                                                          'bit_offset': 0,
                                                                                          'bit_width': 32,
                                                                                          'description': 'MM2S '
                                                                                                         'DMA '
                                                            

                                               'fields': {'Tail_Descriptor_Pointer': {'access': 'read-write',
                                                                                      'bit_offset': 6,
                                                                                      'bit_width': 26,
                                                                                      'description': 'S2MM '
                                                                                                     'DMA '
                                                                                                     'Tail '
                                                                                                     'Descriptor '
                                                                                                     'Pointer '
                                                                                                     'Register'}},
                

                          'parameters': {'C_BASEADDR': '0x00000000',
                                         'C_DM_WIDTH': '4',
                                         'C_DQS_WIDTH': '4',
                                         'C_DQ_WIDTH': '32',
                                         'C_EMIO_GPIO_WIDTH': '64',
                                         'C_EN_EMIO_ENET0': '0',
                                         'C_EN_EMIO_ENET1': '0',
                                         'C_EN_EMIO_PJTAG': '0',
                                         'C_EN_EMIO_TRACE': '0',
                                         'C_FCLK_CLK0_BUF': 'TRUE',
                                         'C_FCLK_CLK1_BUF': 'FALSE',
                                         'C_FCLK_CLK2_BUF': 'FALSE',
                                         'C_FCLK_CLK3_BUF': 'FALSE',
                                         'C_GP0_EN_MODIFIABLE_TXN': '1',
                                         'C_GP1_EN_MODIFIABLE_TXN': '1',


                                         'PCW_DCI_PERIPHERAL_DIVISOR1': '2',
                                         'PCW_DCI_PERIPHERAL_FREQMHZ': '10.159',
                                         'PCW_DDRPLL_CTRL_FBDIV': '21',
                                         'PCW_DDR_DDR_PLL_FREQMHZ': '1050.000',
                                         'PCW_DDR_HPRLPR_QUEUE_PARTITION': 'HPR(0)/LPR(32)',
                                         'PCW_DDR_HPR_TO_CRITICAL_PRIORITY_LEVEL': '15',
                                         'PCW_DDR_LPR_TO_CRITICAL_PRIORITY_LEVEL': '2',
                                         'PCW_DDR_PERIPHERAL_CLKSRC': 'DDR PLL',
                                         'PCW_DDR_PERIPHERAL_DIVISOR0': '2',
                                         'PCW_DDR_PORT0_HPR_ENABLE': '0',
                                         'PCW_DDR_PORT1_HPR_ENABLE': '0',
                                         'PCW_DDR_PORT2_HPR_ENABLE': '0',
                                        

                                         'PCW_MIO_20_SLEW': 'slow',
                                         'PCW_MIO_21_DIRECTION': 'out',
                                         'PCW_MIO_21_IOTYPE': 'LVCMOS 1.8V',
                                         'PCW_MIO_21_PULLUP': 'enabled',
                                         'PCW_MIO_21_SLEW': 'slow',
                                         'PCW_MIO_22_DIRECTION': 'in',
                                         'PCW_MIO_22_IOTYPE': 'LVCMOS 1.8V',
                                         'PCW_MIO_22_PULLUP': 'enabled',
                                         'PCW_MIO_22_SLEW': 'slow',
                                         'PCW_MIO_23_DIRECTION': 'in',
                                         'PCW_MIO_23_IOTYPE': 'LVCMOS 1.8V',
                                         'PCW_MIO_23_PULLUP': 'enabled',
                                         'PCW_MIO_23_SLEW': 'slow',
                                         'PCW_MIO_24_DIRECTION':

                                         'PCW_NOR_SRAM_CS1_T_CEOE': '1',
                                         'PCW_NOR_SRAM_CS1_T_PC': '1',
                                         'PCW_NOR_SRAM_CS1_T_RC': '11',
                                         'PCW_NOR_SRAM_CS1_T_TR': '1',
                                         'PCW_NOR_SRAM_CS1_T_WC': '11',
                                         'PCW_NOR_SRAM_CS1_T_WP': '1',
                                         'PCW_NOR_SRAM_CS1_WE_TIME': '0',
                                         'PCW_NUM_F2P_INTR_INPUTS': '1',
                                         'PCW_OVERRIDE_BASIC_CLOCK': '0',
                                         'PCW_P2F_CAN0_INTR': '0',
                                         'PCW_P2F_CAN1_INTR': '0',
                                         'PCW_P2F_CTI_INTR': '0',
                                         'PCW_P2F_DMAC0_INTR': '0',
                                         'PCW_P2F_DMAC1_INTR': '0',
             

                                         'PCW_UIPARAM_DDR_DQS_2_PROPOGATION_DELAY': '160',
                                         'PCW_UIPARAM_DDR_DQS_3_LENGTH_MM': '0',
                                         'PCW_UIPARAM_DDR_DQS_3_PACKAGE_LENGTH': '113.63',
                                         'PCW_UIPARAM_DDR_DQS_3_PROPOGATION_DELAY': '160',
                                         'PCW_UIPARAM_DDR_DQS_TO_CLK_DELAY_0': '-0.051',
                                         'PCW_UIPARAM_DDR_DQS_TO_CLK_DELAY_1': '-0.006',
                                         'PCW_UIPARAM_DDR_DQS_TO_CLK_DELAY_2': '-0.009',
                                         'PCW_UIPARAM_DDR_DQS_TO_CLK_DELAY_3': '-0.033',
                                         'PCW_UIPARAM_DDR_DQ_0_LENGTH_MM': '32.2',
                                         'PCW_UIPARAM_DDR_DQ_0_PACKAGE_LENGTH': '98.503',
                                         'PCW_UIPARAM_DDR_DQ_0_PROPOGATION_DELAY': '160',
                           

In [31]:
Convolution_design.set_partial_region('block_0')

AttributeError: Could not find IP or hierarchy set_partial_region in overlay